# Building an EVI Timeseries with ODC STAC

This tutorial provides an alternate way to work with Harmonized Landsat Sentinel-2 ([HLS](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/missions/harmonized-landsat-sentinel-2-hls-overview/)) data using [CMR-STAC](https://cmr.earthdata.nasa.gov/stac/docs/index.html) coupled and [OpenDataCube](https://www.opendatacube.org/), which uses the [dask](https://www.dask.org/) and [xarray](https://xarray.dev/) libraries to build a timeseries of data. `OpenDataCube` leverages the STAC metadata to provide values to `xarray` so that data can be loaded lazily with all of the dimensions, coordinate reference system and other relevant attributes. This information along with the cloud-optimized geotiff format of HLS Data means that only the necessary portions of the files required for the desired operation need to be read. On top of this, `dask` is used for parallelization. This workflow is a much faster alternative to the methods used in the [HLS Tutorial](https://github.com/nasa/HLS-Data-Resources/blob/main/python/tutorials/HLS_Tutorial.ipynb), but uses higher level Python libraries, making the code less adaptable to other use-cases. Using these additional Python libraries requires additional dependencies from other resources in this repository. Please use the [Python Environment Setup](#python-environment-setup) section below to set up a compatible Python environment.

## Python Environment Setup

A compatible python environment can be created by following the [Python Environment setup instructions](https://github.com/nasa/VITALS/blob/main/setup/setup_instructions.md), activating that environment and adding the `pystac-client` and `odc-stac` packages:

```
mamba activate lpdaac_vitals
```

```
mamba install -c conda-forge pystac-client odc-stac dask
```

or if you want to set up a separate environment:

```
mamba create -n hls_odc -c conda-forge --yes python=3.12 fiona gdal hvplot geoviews rioxarray rasterio jupyter geopandas earthaccess jupyter_bokeh scikit-image jupyterlab dask odc-stac pystac-client
```

## Tutorial Use Case

This guide examines changes in enhanced vegetation index ([EVI](https://earthobservatory.nasa.gov/features/MeasuringVegetation/measuring_vegetation_4.php)) over an agricultural region in northern California, the same as the [HLS Tutorial: Getting Started with Cloud-Native HLS Data in Python](https://github.com/nasa/HLS-Data-Resources/blob/main/python/tutorials/HLS_Tutorial.ipynb). The goal of the project is to observe HLS-derived mean EVI over these regions without downloading the entirety of the HLS source data in a cloud-friendly, efficient way. In this notebook we will extract an EVI timeseries from Harmonized Landsat Sentinel-2 ([HLS](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/missions/harmonized-landsat-sentinel-2-hls-overview/)) data. First we will search and find data, then we will lazily load only the necessary subsets of these results directly from the cloud and calculate EVI. 

### Background

The Harmonized Landsat Sentinel-2 ([HLS](https://lpdaac.usgs.gov/data/get-started-data/collection-overview/missions/harmonized-landsat-sentinel-2-hls-overview/)) project produces seamless, harmonized surface reflectance data from the Operational Land Imager (OLI) and Multi-Spectral Instrument (MSI) aboard Landsat and Sentinel-2 Earth-observing satellites, respectively. The aim is to produce seamless products with normalized parameters, which include atmospheric correction, cloud and cloud-shadow masking, geographic co-registration and common gridding, normalized bidirectional reflectance distribution function, and spectral band adjustment. This will provide global observation of the Earth’s surface every 2-3 days with 30 meter spatial resolution. One of the major applications that will benefit from HLS is agriculture assessment and monitoring, which is used as the use case for this tutorial.

## Requirements

- A [NASA Earthdata Login](https://urs.earthdata.nasa.gov/) account is required to download the data used in this tutorial. You can create an account at the link provided.

## Learning Objectives

- How to use CMR-STAC to search for HLS data
- How to open and access data using dask and Open Data Cube.

## Data Used

- Daily 30 meter (m) global HLS Sentinel-2 Multi-spectral Instrument Surface Reflectance - [HLSS30.002](https://doi.org/10.5067/HLS/HLSS30.002)  
    - The HLSS30 product provides 30 m Nadir normalized Bidirectional Reflectance Distribution Function (BRDF)-Adjusted Reflectance (NBAR) and is derived from Sentinel-2A and Sentinel-2B MSI data products.  
    - Science Dataset (SDS) layers:  
        - B8A (NIR Narrow)  
        - B04 (Red)  
        - B02 (Blue)   
- Daily 30 meter (m) global HLS Landsat-8 OLI Surface Reflectance - [HLSL30.002](https://doi.org/10.5067/HLS/HLSL30.002)  
    - The HLSL30 product provides 30 m Nadir normalized Bidirectional Reflectance Distribution Function (BRDF)-Adjusted Reflectance (NBAR) and is derived from Landsat-8 OLI data products.  
     - Science Dataset (SDS) layers:  
        - B05 (NIR)  
        - B04 (Red)  
        - B02 (Blue)   

## Tutorial Outline

1. [**Getting Started**](#getstarted)  
    1.1 Import Packages  
    1.2 EarthData Login
    1.3 Set up Dask Client
2. [**Finding HLS Data**](#find)  
3. [**Opening Data with ODC-STAC**](#odcstac)  
    3.1 Apply Scale Factor
4. [**Calculating EVI**](#calcevi)   
5. [**Quality Filtering**](#qualityfilter)  
6. [**Calculating Statistics**](#stats)  

## 1. Getting Started<a id="getstarted"></a>

### 1.1 Import Packages

Import the required packages.

In [ ]:
# Load all the libraries
import os
import numpy as np
import dask.distributed
import pystac_client
import geopandas as gpd
import odc.stac
import xarray as xr
import rasterio as rio
import rioxarray as rxr
import earthaccess
import hvplot.xarray

### 1.2 Earthdata Login Authentication

We will use the `earthaccess` package for authentication. `earthaccess` can either create a a new local `.netrc` file to store credentials or validate that one exists already in you user profile. If you do not have a `.netrc` file, you will be prompted for your credentials and one will be created.

In [ ]:
# Log into earthaccess - ensures creation of .netrc file
earthaccess.login(persist=True)

### 1.3 Set up Dask Client

Here we set up a local `dask` cluster, which will define tasks based on our lazy loaded data and functions, then split those tasks accross our locally available threads or workers to improve process efficiency. You can view the dashboard by clicking the link and see various dashboards and monitor them as you run future cells.

In [ ]:
# Initialize Dask Client
client = dask.distributed.Client()
display(client)

### 1.4 Configure GDAL Options and rio environment

In [ ]:
# Configure odc.stac rio env - requires a .netrc file, sends info to dask client
odc.stac.configure_rio(cloud_defaults=True,
                       verbose=True,
                       client=client,
                       GDAL_DISABLE_READDIR_ON_OPEN='EMPTY_DIR',
                       GDAL_HTTP_COOKIEFILE=os.path.expanduser('~/cookies.txt'),
                       GDAL_HTTP_COOKIEJAR=os.path.expanduser('~/cookies.txt'))

## 2. CMR-STAC Search<a id="find"></a>

To find HLS data, we will use the `pystac_client` python library to search [NASA's Common Metadata Repository SpatioTemporal Asset Catalog (CMR-STAC)](https://cmr.earthdata.nasa.gov/stac/docs/index.html#tag/STAC) for HLS data. We will use a geojson file containing our region of interest (ROI) to search for files that intersect. For this use case, our ROI is an agricultural field in Northern California.

In [5]:
# Open ROI polygon
roi = gpd.read_file('../../data/Field_Boundary.geojson')

Add the collection, datetime range, results limit, and simplify our ROI to a bounding box and store these as search parameters. After defining these, conduct a stac search using the `LPCLOUD` STAC endpoint and return our query as a list of items.

In [6]:
catalog = pystac_client.Client.open("https://cmr.earthdata.nasa.gov/stac/LPCLOUD")
# Define search parameters
search_params = {
    "collections": ["HLSS30_2.0","HLSL30_2.0"],
    "bbox": tuple(list(roi.total_bounds)),
    "datetime": "2021-05-01/2021-09-30",
    "limit": 100,
}
# Perform the search
query = catalog.search(**search_params)
items = query.items()

In [ ]:
items = list(query.items())
print(f"Found: {len(items):d} granules")

We can preview what these results look like.

In [ ]:
items[0]

In [ ]:
items[3]

To stack the data from both Landsat and Sentinel instruments, we need common band names for HLSL30 B5 (NIR) and HLSS30 B8A (NIR). We can simply rename them NIR in the stac results.

In [10]:
# Rename HLSS B8A and HLSL B05 to common band name
for item in items:
    if "HLS.L30" in item.id:
        item.assets["NIR"] = item.assets.pop("B05")
    if "HLS.S30" in item.id:
        item.assets["NIR"] = item.assets.pop("B8A")


In [ ]:
# Confirm this changed the stac results
items[0]

## 3. Opening Data with ODC-STAC<a id="odcstac"></a>

Use the `odc.stac.stac_load` function to load the data from the STAC results lazily into a dataset. To do this, we must provide a crs, a list of STAC results, a tuple of the bands we want to load, an expected resolution, and a dask chunk size.

In [ ]:
# Set CRS and resolution, open lazily with odc.stac
crs = "utm"
ds = odc.stac.stac_load(
    items,
    bands=("B02", "B04","NIR", "Fmask"),
    crs=crs,
    resolution=30,
    chunks={"band":1,"x":512,"y":512},  # If empty, chunks along band dim, 
    #groupby="solar_day", # This limits to first obs per day
)
display(ds)

We can preview the size and shapes of the data we have loaded using the `geobox` function.

In [ ]:
# Show Geobox
ds.odc.geobox

Next, we will clip the data to our ROI using the `rio.clip` function. This will continue to work lazily on the data, so we have yet to actually load any data.

In [14]:
# Clip
ds = ds.rio.clip(roi.geometry.values, roi.crs, all_touched=True)

In [ ]:
# Show Clipped Geobox
ds.odc.geobox

The `scale_factor` information in some of the HLSL30 granules are found in the file metadata, but missing from the Band metadata, meaning this isn't applied automatically. Manually scale each of the data arrays by the scale factor.

In [16]:
# Scale the data
ds.NIR.data = 0.0001 * ds.NIR.data
ds.B04.data = 0.0001 * ds.B04.data
ds.B02.data = 0.0001 * ds.B02.data

Now that we have clipped and scaled our data, we can load the minimal subset of data we need to calculate our EVI over our time period of interest.

In [ ]:
ds.load()

Plot the NIR data to ensure our scaling and clipping worked as expected. We can use the slide bar to scroll through the time dimension showing the 

In [ ]:
# Plot to ensure scaling worked
ds.NIR.hvplot.image(x="x", y="y", groupby="time", cmap="viridis", width=600, height=500, crs='EPSG:32610', tiles='ESRI', rasterize=True)

## 4. Calculate EVI<a id="calcevi"></a>

Now we can build an Enhanced Vegetation Index (EVI) dataset using the EVI formula:

$$
\text{EVI} = G \cdot \frac{\text{NIR} - \text{Red}}{\text{NIR} + C_1 \cdot \text{Red} - C_2 \cdot \text{Blue} + L}
$$

**NIR**: Near-Infrared reflectance.  
**Red**: Reflectance in the red band.  
**Blue**: Reflectance in the blue band, used primarily to correct for aerosol influences.  
**G**: Gain factor (often set to 2.5) that scales the index.  
**C₁**: Coefficient for the aerosol resistance term using the red band (commonly 6).  
**C₂**: Coefficient for the aerosol resistance term using the blue band (commonly 7.5).  
**L**: Canopy background adjustment, which helps to minimize soil brightness influences (commonly 1).  

We will use the typical values adopted for the MODIS algorithm for G, C1, C2 and L. The output values will range from -1 to 1 and indicate strength of vegetation signal, which correlates with biomass.

In [19]:
# Calculate EVI
evi_ds = 2.5 * ((ds.NIR - ds.B04) / (ds.NIR + 6.0 * ds.B04 - 7.5 * ds.B02 + 1.0))

In [20]:
evi_ds = evi_ds.compute()

Now lets visualize our EVI over our ROI. We can scroll through the time-series using the slider bar to the right of the figure.

In [ ]:
evi_ds.hvplot.image(x="x", y="y", groupby="time", cmap="YlGn", clim=(0, 1), crs='EPSG:32610', tiles='ESRI', rasterize=True)

## 5. Quality Masking<a id="qualityfilter"></a>

Lastly, we will apply a function to our EVI dataset, which will mask out pixels based on our bit selection. 

For HLS v2.0 products, all quality information is included in the Fmask layer. This layer includes values corresponding to combinations of bits that represent
different quality descriptions at each location within the scene.  Bits are ordered `76543210` and correspond to the following quality descriptions:  

|Bit Number|Mask Name|Bit Value|Description|
|---|---|---|---|
|7-6|Aerosol<br>Level|11<br>10<br>01<br>00|High<br>Medium<br>Low<br>Clear<br>|
|5|Water|1<br>0|Yes<br>No|
|4|Snow/Ice|1<br>0|Yes<br>No|
|3|Cloud<br>Shadow|1<br>0|Yes<br>No|
|2|Cloud/Shadow<br>Adjacent|1<br>0|Yes<br>No|
|1|Cloud|1<br>0|Yes<br>No|
|0|Cirrus|Reserved|NA|

For example, an 8bit integer 224 converted to binary is 11100000. This would indicate high aerosol (bits 7-6), and water (bit 5) are present in a pixel with that value.

In [22]:
def create_quality_mask(quality_data, bit_nums: list = [1, 2, 3, 4, 5]):
    """
    Uses the Fmask layer and bit numbers to create a binary mask of good pixels.
    By default, bits 1-5 are used.
    """
    mask_array = np.zeros((quality_data.shape[0], quality_data.shape[1]))
    # Remove/Mask Fill Values and Convert to Integer
    quality_data = np.nan_to_num(quality_data.copy(), nan=0).astype(np.int8)
    for bit in bit_nums:
        # Create a Single Binary Mask Layer
        mask_temp = np.array(quality_data) & 1 << bit > 0
        mask_array = np.logical_or(mask_array, mask_temp)
    return mask_array

Use xarray's `apply_ufunc` to apply a function in a vectorized way to our EVI timeseries dataset. `bit_nums` can be provided as a dict to specify which bits to mask out.

In [23]:
quality_mask = xr.apply_ufunc(
    create_quality_mask,
    ds.Fmask,
    kwargs={"bit_nums": [1,2,3,4,5]},
    input_core_dims=[["x", "y"]],
    output_core_dims=[["x", "y"]],
    vectorize=True,
    dask='parallelized',
    output_dtypes=[np.bool],
)

Now use the `where` function to mask out regions corresponding with the bit numbers and visualize the EVI timeseries again.

In [ ]:
evi_ds.where(~quality_mask).hvplot.image(x="x", y="y", groupby="time", cmap="YlGn", clim=(0, 1), crs='EPSG:32610', tiles='ESRI', rasterize=True)

## Contact Info  

Email: LPDAAC@usgs.gov  
Voice: +1-866-573-3222  
Organization: Land Processes Distributed Active Archive Center (LP DAAC)¹  
Website: <https://lpdaac.usgs.gov/>  

¹Work performed under USGS contract G15PD00467 for NASA contract NNG14HH33I. 